# 1. Motivation
In Machine Learning, there are commonly two problems (functions) that need to be optimized:

$$\begin{alignat}{3}
f &:\text{parameter} &&\rightarrow\text{loss} \qquad &&(1) \\
f &:\text{hyperparameter} &&\rightarrow\text{score}\qquad &&(2)
\end{alignat}$$

The first problem is nothing but the training/fitting step; the loss function $\mathcal{L}$ to be minimized is known and more importantly, *differentiable*. Therefore, a gradient-based method such as Gradient Descent can be used to estimate model parameters.

The second one (also known as hyperparameter tuning) does not have a mathematical form and thus cannot be solved using Gradient Descent. The optimal set of hyperparameters is usually found either by Data Scientists using rule of thumb or by searching methods. However, each trial is expensive and we must find the optimal configurations with as few trials as possible.

Scoring strategies in Machine Learning can be either *higher is better* (R2, AUC, F1,...) or *lower is better* (RMSE, MAE,...). To make things consistant, we assume all evaluation metrics are higher better.

## 2.1. Exhausted search
There are two popular methods of searching hyperparameters, Grid Search and Random Search. The idea of each method are already explained clearly in the image below, where the red area shows how much each hyperparameter contributes to model score.

<img src='image/grid_random_search.png' style='width:450px; margin:20px auto;'>

Grid Search goes to every possible combinations, and thus it will revisit a parameter value multiple times. This makes Grid Search have a low coverage and is very expensive, especially for algorithms with a huge number of hyperparameters such as XGBoost and LightGBM.

Random Search is a more efficient method, it randomly takes a number combinations to train models. With the same number of iterations provided, Random Search can cover a wider range of values, and thus it is able to reach the optimal value that Grid Search cannot. Another advantage is that you can include less important hyperparameters in your search without increasing the number of iterations.

## 2.2. Sequential search
The searching methods are actually wasting a lot of useful information from previous trials. In order to take advantages of historical information, SMBO (**S**equential **M**odel-**B**ased **O**ptimization) comes to the rescue. The basic idea of SBMO can be simply summarized into two steps:
- Build a probabilistic *surrogate model* of the black-box function $f:\text{hyperparameter}\rightarrow\text{score}$ using results from observed expensive trials
- According to the surrogate model, choose the most promising hyperparameters set to evaluate in the next expensive trial

### Algorithm
*Input*:
- A domain, or search space of hyperparameters
- $f$, the black-box function to be optimized, it maps hyperpameters (denoted $x$) to model score (denoted $y=f(x)$) and is very expensive to evaluate
- $T$, the number of trials budget
- $\mathcal{S}$, a surrogate model which takes finished trials as input then returns a distribution of model score for each unobserved trial. Popular options GP (Gaussian Process) and TPE (Tree-structered Parzen Estimators) will be disussed in the next sections.
- $\mathcal{A}$, an acquisition function for deciding where the next trial should locate at

*Step 1*: Create a set $\mathcal{H}$ for storing historical trials $(x,y)$ with a randomly initialized trial $(x_0,y_0)$.

*Step 2*: For each trial $t$ for $t=1,2,\dots,T$:
- Fit the surrogate model on $\mathcal{H}$ to get $\mathcal{S}_t$
- Compute the acquisition function $\mathcal{A}_t(x)$ using $\mathcal{S}_t$ and $\mathcal{H}$
- Evaluate the most promising query point $x_t=\arg\max\mathcal{A}_t(x)$
- Train the expensive Machine Learning model using hyperparameters $x_t$ and compute its score $y_t=f(x_t)$
- Append the result $(x_t,y_t)$ to $\mathcal{H}$

### Acquisition functions
Acquisition function is a very important component of SMBO, it defines the strategy to select the next set of hyperparameters to be used in training Machine Learning model. Acqusition functions control the balance between *exploitation* and *exploration*. In this section, we learn about 4 most popular options, using these notations:
- $x^\star,y^\star$ are the best hyperparameters so far in $\mathcal{H}$ and the associated model score
- $\varphi(\cdot)$ and $\psi(\cdot)$ indicate the PDF and CDF, respectively
- $\mu_y$ and $\sigma_y$ indicate the mean and standard deviation of model score according to the surrogate model

***P**robability of **I**mprovement*. The idea of PI is very simple, it measures, at each unused configuration $x$, the probability that the corresponding score $y$ is higher than the current best score $y^\star$. A small positive offset $\epsilon$ is added to maintain the trade-off between *exploration* and *exploitation*.

$$\text{PI}=\text{Prob }(y>y^\star+\epsilon)$$

The intuition behind $\epsilon$ is all about *uncertainty*. For certain areas, the distribution of model scores are clustered around the mean, so that only a small $\epsilon$ penalizes their PI a lot. For this reason, higher values of $\epsilon$ favor exploration; but if too high, the behaviour will be a lot like [Active Learning](<https://en.wikipedia.org/wiki/Active_learning_(machine_learning)>). A balanced $\epsilon$ will make SMBO explore *just enough* and spend trials effectively on exploitation.

***E**xpected **I**mprovement*

$$\text{EI}=\int_{-\infty}^{\infty} {(y-y^\star-\epsilon)\,\varphi(z)\,\text{d}z}$$

*Upper Confidence Bound*

*Thomson Sampling*

# 2. Bayesian Optimization
A SBMO using Gaussian Process as the surrogate function is called **B**ayesian **O**ptimization (BO), since Gaussian Process use Bayes' rule to update the posterior distribution.

# 3. Parzen Estimators

# 4. Implementation

# References
- *papers.nips.cc* - [Algorithms for Hyper-parameters Optimization](https://papers.nips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf)
- *openreview.net* - [Hyperband: Bandit-based configuration evaluation for hyperparameter optimization](https://openreview.net/pdf?id=ry18Ww5ee)
- *medium.com* - [A Conceptual Explanation of Bayesian Hyperparameter Optimization for Machine Learning](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)
- *medium.com* - [Hyper-parameter optimization algorithms: a short review](https://medium.com/criteo-engineering/hyper-parameter-optimization-algorithms-2fe447525903)
- *krasserm.github.io* - [Bayesian Optimization](https://krasserm.github.io/2018/03/21/bayesian-optimization/)
- *distill.pub* - [Exploring Bayesian Optimization](https://distill.pub/2020/bayesian-optimization/)

---
*&#9829; By Quang Hung x Thuy Linh &#9829;*